In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import json
import joblib

In [4]:
ratings = pd.read_csv(r"C:\Users\jason\OneDrive\Documents\Python QT\ML_Model\data\ratings.csv")
books = pd.read_csv(r"C:\Users\jason\OneDrive\Documents\Python QT\ML_Model\data\books.csv")

In [7]:
user_item_matrix = ratings.pivot_table(index="user_id", columns="book_id", values="rating").fillna(0)

Index(['book_id', 'authors', 'title', 'average_rating', 'ratings_count'], dtype='object')

Create the matrix for model exploration.

In [ ]:
scaler = StandardScaler()
scaled_matrix = scaler.fit_transform(user_item_matrix)

Normalizes the data.

In [ ]:

k = 8
model = KMeans(n_clusters=k, random_state=42),
model.fit(scaled_matrix)

Trains model

In [ ]:
user_clusters = pd.DataFrame({
    "user_id": user_item_matrix.index,
    "cluster": model.labels_
})

Attach a cluster to each user.

In [ ]:
ratings_with_clusters = ratings.merge(user_clusters, on="user_id")

Merge with ratings.

In [ ]:
cluster_book_map = {}
for cluster_id in range(k):
    cluster_ratings = ratings_with_clusters[ratings_with_clusters["cluster"] == cluster_id]
    book_stats = (
        cluster_ratings.groupby("book_id")["rating"]
        .agg(["mean", "count"])
        .reset_index()
    )
    top_books = book_stats[book_stats["mean"] >= 4.5]["book_id"].tolist()
    cluster_book_map[str(cluster_id)] = top_books

Finds the best books within each cluster.

In [ ]:
book_meta_dict = books.set_index("book_id").to_dict(orient="index")

Prepares books meta-data for Django.

In [ ]:

with open("output/cluster_books.json", "w") as f:
    json.dump(cluster_book_map, f)
with open("output/book_meta.json", "w") as f:
    json.dump(book_meta_dict, f)

Exports the best books and associated metadata.